# CAE

Let's use a CAE to generate a good representation of waterbodies.

## Setup

### Load modules

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import scipy.optimize as opt
import matplotlib.colors
import matplotlib.patches
from tqdm.notebook import tqdm
import sklearn.cluster
import sklearn.decomposition
import scipy.ndimage.filters
import scipy.cluster
import matplotlib.animation
import datacube

import sys
sys.path.insert(1, '../Scripts')
import dea_plotting

import keras

%matplotlib widget

Using TensorFlow backend.
/g/data/v10/public/modules/dea-env/20200713/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/g/data/v10/public/modules/dea-env/20200713/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/g/data/v10/public/modules/dea-env/20200713/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

### Load the data

This was generated in WaterbodyClustering.ipynb.

In [2]:
history = np.load('history_murray_full_norivers.npy')
times = np.load('time_axis_murray_full_norivers.npy').astype('datetime64[D]')
waterbodies = gpd.read_file('waterbodies_murray_norivers.geojson')

We will work with the last five years of data, not blurred since we have convolutions.

In [198]:
input_size = 2048
data = history[:, -input_size:]
times_ = times[-input_size:]

In [199]:
mean = data.mean(axis=0)

## Keras time

In [200]:
from keras.layers import Input, Dense, Conv1D, MaxPooling1D, UpSampling1D
from keras.models import Model
from keras import backend as K

In [201]:
input_img = Input(shape=(input_size, 1))

In [202]:
x = Conv1D(8, 7, activation='relu', padding='same')(input_img)
x = MaxPooling1D(4, padding='same')(x)

In [203]:
x = Conv1D(8, 3, activation='relu', padding='same')(x)
x = MaxPooling1D(4, padding='same')(x)
x = Conv1D(4, 3, activation='relu', padding='same')(x)
x = Conv1D(4, 3, activation='relu', padding='same')(x)
encoded = MaxPooling1D(4, padding='same')(x)

In [204]:
x = Conv1D(4, 3, activation='relu', padding='same')(encoded)
x = UpSampling1D(4)(x)
x = Conv1D(8, 3, activation='relu', padding='same')(x)
x = UpSampling1D(4)(x)
x = Conv1D(8, 3, activation='relu', padding='same')(x)
x = Conv1D(8, 3, activation='relu', padding='same')(x)
x = UpSampling1D(4)(x)
decoded = Conv1D(1, 2, activation='sigmoid', padding='same')(x)

In [205]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='mse')

In [206]:
data_ = data.reshape(data.shape + (1,)) / 100

In [207]:
autoencoder.fit(data_, data_,
                epochs=50,
                batch_size=128,
                shuffle=True)

Epoch 1/50
9081/9081 [==============================] - 16s 2ms/step - loss: 0.0981
Epoch 2/50
9081/9081 [==============================] - 8s 905us/step - loss: 0.0233
Epoch 3/50
9081/9081 [==============================] - 8s 883us/step - loss: 0.0166
Epoch 4/50
9081/9081 [==============================] - 8s 900us/step - loss: 0.0152
Epoch 5/50
9081/9081 [==============================] - 8s 916us/step - loss: 0.0137
Epoch 6/50
9081/9081 [==============================] - 8s 888us/step - loss: 0.0130
Epoch 7/50
9081/9081 [==============================] - 8s 916us/step - loss: 0.0125
Epoch 8/50
9081/9081 [==============================] - 8s 909us/step - loss: 0.0122
Epoch 9/50
9081/9081 [==============================] - 8s 895us/step - loss: 0.0117
Epoch 10/50
9081/9081 [==============================] - 8s 903us/step - loss: 0.0115
Epoch 11/50
9081/9081 [==============================] - 8s 900us/step - loss: 0.0116
Epoch 12/50
9081/9081 [==============================] - 8s 911u

In [208]:
encoder = Model(input_img, encoded)

In [209]:
enc_ = encoder.predict(data_)

In [210]:
pca = sklearn.manifold.TSNE(n_components=2)

In [211]:
pca_f = pca.fit_transform(enc_.reshape(data.shape[0], -1))

In [216]:
plt.figure()
c = (pca_f[:, 0] > 62) & (pca_f[:, 1] > -20)
plt.scatter(*pca_f.T, s=1, c=c)

/g/data/v10/public/modules/dea-env/20200713/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

A single blob keeps showing up in the tSNE for autoencoded stuff. What is it? I suspect it's like "dry all the time" or something. Let's see.

In [220]:
dc = datacube.Datacube()
def cluster_sample(cluster_mask):
    das_1 = []
    cluster_1_sample = np.random.choice(cluster_mask.nonzero()[0], size=9, replace=False)
    for k in cluster_1_sample:
        waterbody = waterbodies.iloc[k]
        da = dc.load('ls8_nbart_geomedian_annual', measurements=['red', 'green', 'blue'],
                     geopolygon=datacube.utils.geometry.Geometry(waterbody.geometry.buffer(1500), crs='EPSG:3577'), time='2019-01')
        das_1.append(da.isel(time=0))
    plt.figure(figsize=(10, 10))
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        dea_plotting.rgb(das_1[i], bands=['red', 'green', 'blue'], ax=ax)
        geometry = waterbodies.iloc[cluster_1_sample[i]].geometry
        patch = matplotlib.patches.Polygon(geometry.exterior, edgecolor='white', facecolor='#AAAAFF55')
        ax.add_patch(patch)
        ax.set_title('')
        ax.axis('off')
        ys = data[cluster_1_sample[i]]
        ys = ys / max(ys) * 0.5
        xs = np.linspace(0, 1, len(ys))
        points = np.stack([xs, ys]).T
        tf_sp = ax.transData.inverted().transform(ax.transAxes.transform(points))
        plt.plot(tf_sp[:, 0], tf_sp[:, 1], c='white')
    

In [221]:
cluster_sample(c)

/g/data/v10/public/modules/dea-env/20200713/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # Remove the CWD from sys.path while we load stuff.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/g/data/v10/public/modules/dea-env/20200713/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/g/data/v10/public/modules/dea-env/20200713/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/g/data/v10/public/modules/dea-env/20200713/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/g/data/v10/public/modules/dea-env/20200713/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/g/data/v10/public/modules/dea-env/20200713/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/g/data/v10/public/modules/dea-env/20200713/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide


Yeah it's dry stuff.

In [241]:
plt.figure()
plt.plot(times_, data_[500])
plt.plot(times_, autoencoder.predict(data_[500][None, :]).ravel())

/g/data/v10/public/modules/dea-env/20200713/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## RNN

One way to encode sequential data while taking advantage of its (temporal) structure is to use an RNN. These work absurdly well on sequences, even just simple ones. Let's build a dead-simple RNN in Keras and get it to predict water sequences. I briefly thought that it might be a good idea to pass the gradient in as well, but then it becomes really easy to guess what the value should be (just multiply the gradient by the time step...).

In [226]:
from keras.models import Sequential  
from keras.layers.core import Dense, Activation  
from keras.layers.recurrent import LSTM

In [238]:
data_diffs = np.diff(data / 100)

In [270]:
time_window = 14  # days
Xs = []
ys = []
for row in tqdm(data_diffs):
    for start in range(0, len(row) - time_window - 1, 2):
        Xs.append(row[start:start + time_window])
        ys.append(row[start + time_window])

In [271]:
Xs = np.array(Xs).reshape(len(Xs), time_window, 1)
ys = np.array(ys)

In [272]:
Xs_train = Xs[::100]
ys_train = ys[::100]

In [273]:
model = Sequential()
n_hidden = 25
model.add(LSTM(n_hidden, input_shape=(time_window, 1), return_sequences=False))
model.add(Dense(1, input_shape=(n_hidden,)))
model.add(Activation('tanh'))
model.compile(loss='mse', optimizer='rmsprop')

In [274]:
history = model.fit(Xs_train, ys_train, epochs=2, batch_size=128, verbose=1, shuffle=True)

Epoch 1/2
92263/92263 [==============================] - 13s 138us/step - loss: 5.9777e-04
Epoch 2/2
92263/92263 [==============================] - 7s 79us/step - loss: 5.9325e-04


In [288]:
predictions = model.predict(Xs[:1000])

In [290]:
plt.figure()
plt.plot(ys[:1000])
plt.plot(predictions)

/g/data/v10/public/modules/dea-env/20200713/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …